<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/07_document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 単語埋め込みを使った文書分類
* 今回は、fastTextのような学習済みの単語埋め込みは使わない。
* 単語埋め込み自体の学習も、ネットワークの学習と同時におこなう。
* IMDbデータの準備も、`torch.torchtext`を使っておこなう。
* 参考資料
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

## データをどう扱うか
* ネットワークへの入力は、単語埋め込みを、単語の出現順どおりに並べた列にする。
 * ミニバッチは[ミニバッチのなかでの最大文書長, ミニバッチのサイズ, 単語埋め込み次元数]という形のテンソルになる。
* そして、前向き計算のなかではじめて、単語埋め込みの平均をとることにする。
 * `.mean(0)`と、軸0で平均をとることになる。


## 07-00 Google Colabのランタイムのタイプを変更する
* Google ColabのランタイムのタイプをGPUに変更しておこう。
 * 上のメニューの「ランタイム」→「ランタイムのタイプを変更」→「ハードウェア　アクセラレータ」から「GPU」を選択

## 07-01 torchtextを使ってIMDbデータを読み込む
* ここでIMDbデータセットの読み込みにつかう`torchtext.datasets`については、下記を参照。
 * https://torchtext.readthedocs.io/en/latest/datasets.html

### 実験の再現性確保のための設定など
* torch.backends.cudnn.deterministicをTrueにするのは、こうしないと、GPU上での計算が毎回同じ値を与えないため。

In [1]:
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
from torchtext.data import Field, LabelField, BucketIterator

SEED = 123

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### torchtextのフィールド
* TEXTフィールドと、LABELフィールドという２種類のFieldオブジェクトのインスタンスを作る。
* TEXTフィールドは、テキストの前処理の仕方を決めておくのに使う。
 * tokenizerは、デフォルトでは単にstring型のsplitメソッドを適用するだけになる。これは高速だが、tokenizationとしては雑。
* LABELフィールドは、ラベルの前処理に使う。

In [2]:
TEXT = Field(tokenize="spacy")
LABEL = LabelField()

### IMDbデータセットをダウンロードした後、前処理しつつ読み込む
* ダウンロードはすぐ終わるが、解凍に少し時間がかかる。
* また、TEXTフィールドでspaCyのtokenizationを使うように設定したので、少し時間がかかる。
 * string型のsplitメソッドでtokenizeすると、時間はあまりかからない。（そのかわり、やや雑なtokenizationになる。）

In [3]:
train_valid_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

### 最初の文書を見てみる
（ちなみに、`vars`関数は、モジュール、クラス、インスタンス、あるいはそれ以外の`__dict__`属性を持つオブジェクトの、`__dict__`属性を辞書として返す組み込み関数。）

In [4]:
print(vars(train_valid_data.examples[0]))

{'text': ['McConaughey', 'in', 'a', 'horror', '/', 'thriller', '?', 'I', 'had', 'to', 'see', 'this', '.', 'I', 'was', 'pleasantly', 'surprised.<br', '/><br', '/>The', 'plot', 'is', 'told', 'in', 'flashback', 'mode', ',', 'and', 'it', 'concerns', 'an', 'otherwise', 'normal', 'and', 'happy', 'family', 'of', 'three', 'going', 'through', 'a', 'very', 'bizarre', 'predicament', '.', 'I', 'ca', "n't", 'say', 'much', 'more', 'without', 'spoiling', 'the', 'whole', 'movie', ',', 'sorry', '.', 'Just', 'know', 'that', 'if', 'you', 'decide', 'to', 'watch', 'it', ',', 'you', "'ll", 'be', ',', 'in', 'the', 'very', 'least', ',', 'surprised.<br', '/><br', '/>All', 'the', 'main', 'players', 'are', 'very', 'good', '.', 'Bill', 'Paxton', 'did', 'a', 'great', 'job', 'directing', 'those', 'kids', ',', 'and', 'his', 'acting', 'is', 'awesome', '.', 'McConaughey', "'s", 'acting', 'is', 'solid', 'throughout', 'and', 'fits', 'the', 'bill', 'perfectly.<br', '/><br', '/>This', 'movie', 'challenges', 'you', 'to', '

In [5]:
print(train_valid_data.examples[0].text)

['McConaughey', 'in', 'a', 'horror', '/', 'thriller', '?', 'I', 'had', 'to', 'see', 'this', '.', 'I', 'was', 'pleasantly', 'surprised.<br', '/><br', '/>The', 'plot', 'is', 'told', 'in', 'flashback', 'mode', ',', 'and', 'it', 'concerns', 'an', 'otherwise', 'normal', 'and', 'happy', 'family', 'of', 'three', 'going', 'through', 'a', 'very', 'bizarre', 'predicament', '.', 'I', 'ca', "n't", 'say', 'much', 'more', 'without', 'spoiling', 'the', 'whole', 'movie', ',', 'sorry', '.', 'Just', 'know', 'that', 'if', 'you', 'decide', 'to', 'watch', 'it', ',', 'you', "'ll", 'be', ',', 'in', 'the', 'very', 'least', ',', 'surprised.<br', '/><br', '/>All', 'the', 'main', 'players', 'are', 'very', 'good', '.', 'Bill', 'Paxton', 'did', 'a', 'great', 'job', 'directing', 'those', 'kids', ',', 'and', 'his', 'acting', 'is', 'awesome', '.', 'McConaughey', "'s", 'acting', 'is', 'solid', 'throughout', 'and', 'fits', 'the', 'bill', 'perfectly.<br', '/><br', '/>This', 'movie', 'challenges', 'you', 'to', 'think', '

In [6]:
print(train_valid_data.examples[0].label)

pos


### テストセット以外の部分を訓練データと検証データに分ける

In [7]:
train_data, valid_data = train_valid_data.split(split_ratio=0.8, random_state = random.seed(SEED))

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


### データセットの語彙とラベルを作る
* TEXTラベルのほうでは、最大語彙サイズを指定する。

In [9]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

なぜ語彙サイズが25,000ではなく25,002なのかについては、少し下の説明を参照。

In [10]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


### 出現頻度順で上位２０単語を見てみる

In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 232103), (',', 220515), ('.', 189706), ('and', 125577), ('a', 125352), ('of', 115040), ('to', 107448), ('is', 86946), ('in', 70203), ('I', 62211), ('it', 61179), ('that', 56449), ('"', 50102), ("'s", 49672), ('this', 48320), ('-', 42239), ('/><br', 40900), ('was', 40224), ('as', 34945), ('with', 34442)]


### 単語ID順に最初の１０単語を見てみる
* IDのうち、0と1は、未知語とパディング用の単語という特殊な単語に割り振られている。
 * 未知語は`<unk>`という特殊な単語に置き換えられる。これのIDが0。
 * パディングとは、長さが不揃いの複数の文書を同じミニバッチにまとめるとき、すべての文書の長さを無理やりそろえるため、文書末尾に特殊な単語（元々の語彙にない、人工的に用意した単語）を追加すること。
 * パディング用の単語が`<pad>`になっているのは、上のほうで使ったFieldクラスのインスタンスを作るときのデフォルトの値がこの`<pad>`になっているため。

In [12]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


### ラベルのほうのIDを確認する
* こちらはnegとposに対応する２つのIDしかない。

In [13]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fccad032ea0>, {'neg': 0, 'pos': 1})


### ミニバッチを取り出すためのiteratorを作る
* ミニバッチのサイズを指定する。
 * ミニバッチのサイズは、性能を出すためにチューニングすべきハイパーパラメータのひとつ。

In [14]:
BATCH_SIZE = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size=BATCH_SIZE,
    device=device)

### 試しに検証セットのiteratorを回してミニバッチをすべて取得して個数を数えてみる

In [15]:
i = 0
for batch in valid_iterator:
  i += 1
  continue
print(f'We have {i} mini-batches in validation set.')

We have 50 mini-batches in validation set.


### ミニバッチの中身を見てみる
* 上記のループを抜けたあとには、変数batchには検証セットの最後のミニバッチが代入されている。
* そこで、この最後のミニバッチのshapeを確認する。

### ミニバッチの形を確認する
* ミニバッチの形は、[ミニバッチに含まれる最長文書の文書長, ミニバッチのサイズ]になっていることに注意！
 * ミニバッチのサイズが最初に来ているのではない！
* [ミニバッチのサイズ, ミニバッチに含まれる最長文書の文書長]という形にしたいときは、テキストのfieldを作るときにに以下のようにする。

__`TEXT = data.Field(tokenize="spacy", batch_first=True)`__

In [16]:
batch.text.shape

torch.Size([1989, 100])

このミニバッチに含まれる文書のうち、最初のものの単語ID列と、先頭100個のIDを単語に戻したものを表示させてみる。

In [17]:
print(batch.text[:, 0])
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:100, 0]]))

tensor([1075,  223,   31,  ...,  857,   24,    4], device='cuda:0')
By now you 've probably heard a bit about the new Disney dub of Miyazaki 's classic film , Laputa : Castle In The Sky . During late summer of 1998 , Disney released " Kiki 's <unk> Service " on video which included a preview of the Laputa dub saying it was due out in " 1999 " . It 's obviously way past that year now , but the dub has been finally completed . And it 's not " Laputa : Castle In The Sky " , just " Castle In The Sky " for the dub


最後の文書の末尾は「1」で埋められていることが分かる。

この1は、パディング用単語のIDだったことを想起されたい。

In [18]:
print(batch.text[:, BATCH_SIZE-1])

tensor([  25, 8329,  928,  ...,    1,    1,    1], device='cuda:0')


ミニバッチに含まれる文書の長さを調べると、文書が文書長の降順に並べられていることが分かる。

In [19]:
(batch.text != 1).sum(0)

tensor([1989, 1446, 1411, 1387, 1332, 1246, 1206, 1193, 1191, 1191, 1183, 1181,
        1177, 1175, 1163, 1162, 1161, 1155, 1155, 1155, 1150, 1146, 1145, 1143,
        1139, 1136, 1126, 1125, 1123, 1122, 1122, 1122, 1122, 1120, 1113, 1111,
        1103, 1100, 1095, 1089, 1089, 1083, 1080, 1079, 1077, 1070, 1070, 1065,
        1064, 1052, 1051, 1047, 1045, 1041, 1041, 1035, 1035, 1035, 1032, 1032,
        1025, 1017, 1016, 1016, 1012, 1004, 1002, 1000,  991,  988,  979,  976,
         975,  971,  968,  965,  964,  964,  963,  958,  954,  952,  950,  950,
         949,  949,  947,  941,  940,  933,  929,  929,  927,  926,  925,  924,
         921,  920,  915,  906], device='cuda:0')

## 07-02 MLPによる文書分類の準備
* 今回は、ごく簡単なMLPで文書分類をする。
* 文書中の全単語トークンの埋め込みベクトルの平均を、MLPの入力とする。
 * 当然、語順の情報は使われない。
 * つまり、bag-of-wordsモデルになっている。

### 定数の設定
* 単語埋め込みベクトルの次元数は128にする。

In [20]:
INPUT_DIM = len(TEXT.vocab)
NUM_CLASS = len(LABEL.vocab)
EMBED_DIM = 128
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

### モデルを定義する前にPyTorchの単語埋め込みがどんなものかを見てみる

以下のように、語彙サイズと埋め込みの次元数を指定しつつ、torch.nn.Embeddingのインスタンスを作ればよい。

In [21]:
embed = nn.Embedding(INPUT_DIM, EMBED_DIM, padding_idx=PAD_IDX)

パディング用の単語の埋め込みはゼロベクトルになる。

In [22]:
print(embed(torch.tensor([[2,3],[0,1]])))

tensor([[[ 2.3862e-01,  1.4106e-01, -1.3354e+00, -2.9340e+00,  1.1411e-01,
          -1.2072e+00, -3.0083e-01,  1.4274e-01, -1.3027e+00, -4.9187e-01,
          -2.1429e+00,  9.4881e-01, -5.6842e-01, -6.4643e-02,  6.6467e-01,
          -2.7836e+00,  1.1366e+00,  9.0886e-01,  9.4943e-01,  2.6565e-02,
          -9.2207e-01,  7.0338e-01, -3.6590e-01, -1.9654e-01, -9.2071e-01,
           3.1535e-01, -2.1734e-02,  3.4414e-01,  2.2710e-01, -4.5969e-01,
          -6.1831e-01,  2.4612e-01, -4.0549e-01, -8.3681e-01,  1.2277e+00,
          -4.2971e-01, -2.2121e+00, -3.7802e-01,  9.8382e-01, -1.0895e+00,
           2.0171e-01,  2.2145e-02, -1.7753e+00, -7.4896e-01,  2.7808e-01,
          -9.6208e-01, -4.2228e-01, -1.1036e+00,  2.4727e-01,  1.4549e+00,
          -2.8351e-01, -3.7675e-01, -3.0577e-02, -8.9448e-02, -1.9652e-01,
          -9.7133e-01,  9.0046e-01, -2.5233e-01,  1.0669e+00, -2.9846e-01,
           8.5576e-01,  1.6098e+00, -1.1893e+00,  1.1677e+00,  3.2765e-01,
          -8.3307e-01, -1

### モデルの定義
* MLP（多層パーセプトロン）だが、入り口に単語埋め込み層が挿入されている。

In [23]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.fc1.weight.data.uniform_(-initrange, initrange)
    self.fc1.bias.data.zero_()
    self.fc2.weight.data.uniform_(-initrange, initrange)
    self.fc2.bias.data.zero_()
    self.fc3.weight.data.uniform_(-initrange, initrange)
    self.fc3.bias.data.zero_()

  def forward(self, text):
    x = self.embed(text)
    x = x.mean(0) 
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

### モデルを作る
* モデル（のインスタンス）をGPUに移動させている点に注意。

In [24]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX).to(device)

### 損失関数とoptimizerとschedulerを作る

In [25]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

### 訓練用の関数
* 最初の`model.train()`に注意。こうやって、モデルを訓練モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [26]:
def train(data_iterator, model, optimizer, scheduler, criterion):

  model.train()

  train_loss = 0
  train_acc = 0
  for batch in data_iterator:
    optimizer.zero_grad()
    text, cls = batch.text, batch.label
    output = model(text)
    loss = criterion(output, cls)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    train_acc += (output.argmax(1) == cls).sum().item()

  scheduler.step()

  data_len = len(data_iterator.dataset)
  return train_loss / data_len, train_acc / data_len

### 評価用の関数
* 最初の`model.eval()`に注意。こうやって、モデルを評価モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [27]:
def test(data_iterator, model, criterion):

  model.eval()

  loss = 0
  acc = 0
  for batch in data_iterator:
    text, cls = batch.text, batch.label
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, cls)
      loss += loss.item()
      acc += (output.argmax(1) == cls).sum().item()

  data_len = len(data_iterator.dataset)
  return loss / data_len, acc / data_len

## 07-03 分類器の訓練と開発セットでの評価

In [28]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 4 seconds
	Loss: 0.0042(train)	|	Acc: 80.6%(train)
	Loss: 0.0001(valid)	|	Acc: 88.3%(valid)
Epoch: 2  | time in 0 minutes, 4 seconds
	Loss: 0.0016(train)	|	Acc: 94.2%(train)
	Loss: 0.0001(valid)	|	Acc: 88.5%(valid)
Epoch: 3  | time in 0 minutes, 4 seconds
	Loss: 0.0007(train)	|	Acc: 97.8%(train)
	Loss: 0.0001(valid)	|	Acc: 87.3%(valid)
Epoch: 4  | time in 0 minutes, 4 seconds
	Loss: 0.0003(train)	|	Acc: 98.9%(train)
	Loss: 0.0002(valid)	|	Acc: 87.1%(valid)
Epoch: 5  | time in 0 minutes, 4 seconds
	Loss: 0.0002(train)	|	Acc: 99.4%(train)
	Loss: 0.0002(valid)	|	Acc: 86.5%(valid)
Epoch: 6  | time in 0 minutes, 4 seconds
	Loss: 0.0001(train)	|	Acc: 99.7%(train)
	Loss: 0.0004(valid)	|	Acc: 86.6%(valid)
Epoch: 7  | time in 0 minutes, 4 seconds
	Loss: 0.0001(train)	|	Acc: 99.8%(train)
	Loss: 0.0003(valid)	|	Acc: 86.6%(valid)
Epoch: 8  | time in 0 minutes, 4 seconds
	Loss: 0.0000(train)	|	Acc: 99.9%(train)
	Loss: 0.0004(valid)	|	Acc: 87.6%(valid)
Epoch: 9  | time

## 07-04 再検討
* 訓練データ上での分類精度が100%になってしまっている。明らかにオーバーフィッティング。

### ドロップアウトを使う
* モデルのインスタンスを作るときにdropoutの確率を引数pで指定できるようにする。

In [29]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx, p=0.0):
    super(EmbedTextSentiment, self).__init__()
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.dropout = nn.Dropout(p=p)
    self.fc1 = nn.Linear(embed_dim, 500)
    self.fc2 = nn.Linear(500, 100)
    self.fc3 = nn.Linear(100, num_class)
    self.init_weights()

  def init_weights(self):
    initrange = 0.5
    self.fc1.weight.data.uniform_(-initrange, initrange)
    self.fc1.bias.data.zero_()
    self.fc2.weight.data.uniform_(-initrange, initrange)
    self.fc2.bias.data.zero_()
    self.fc3.weight.data.uniform_(-initrange, initrange)
    self.fc3.bias.data.zero_()

  def forward(self, text):
    x = self.dropout(self.embed(text))
    x = x.mean(0)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [30]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [31]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 4 seconds
	Loss: 0.0048(train)	|	Acc: 76.6%(train)
	Loss: 0.0001(valid)	|	Acc: 87.8%(valid)
Epoch: 2  | time in 0 minutes, 4 seconds
	Loss: 0.0024(train)	|	Acc: 90.4%(train)
	Loss: 0.0001(valid)	|	Acc: 88.5%(valid)
Epoch: 3  | time in 0 minutes, 4 seconds
	Loss: 0.0016(train)	|	Acc: 93.9%(train)
	Loss: 0.0001(valid)	|	Acc: 88.9%(valid)
Epoch: 4  | time in 0 minutes, 4 seconds
	Loss: 0.0011(train)	|	Acc: 95.9%(train)
	Loss: 0.0001(valid)	|	Acc: 88.9%(valid)
Epoch: 5  | time in 0 minutes, 4 seconds
	Loss: 0.0009(train)	|	Acc: 96.7%(train)
	Loss: 0.0002(valid)	|	Acc: 86.8%(valid)
Epoch: 6  | time in 0 minutes, 4 seconds
	Loss: 0.0007(train)	|	Acc: 97.3%(train)
	Loss: 0.0002(valid)	|	Acc: 88.5%(valid)
Epoch: 7  | time in 0 minutes, 4 seconds
	Loss: 0.0005(train)	|	Acc: 98.0%(train)
	Loss: 0.0002(valid)	|	Acc: 88.4%(valid)
Epoch: 8  | time in 0 minutes, 4 seconds
	Loss: 0.0005(train)	|	Acc: 98.3%(train)
	Loss: 0.0002(valid)	|	Acc: 88.4%(valid)
Epoch: 9  | time

### L２正則化を使う
* optimizerのweight_decayパラメータを0より大きな値にする。

In [32]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [33]:
N_EPOCHS = 20
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 4 seconds
	Loss: 0.0067(train)	|	Acc: 62.3%(train)
	Loss: 0.0002(valid)	|	Acc: 70.0%(valid)
Epoch: 2  | time in 0 minutes, 4 seconds
	Loss: 0.0052(train)	|	Acc: 74.9%(train)
	Loss: 0.0002(valid)	|	Acc: 80.9%(valid)
Epoch: 3  | time in 0 minutes, 4 seconds
	Loss: 0.0045(train)	|	Acc: 79.5%(train)
	Loss: 0.0002(valid)	|	Acc: 83.6%(valid)
Epoch: 4  | time in 0 minutes, 4 seconds
	Loss: 0.0041(train)	|	Acc: 82.6%(train)
	Loss: 0.0001(valid)	|	Acc: 85.3%(valid)
Epoch: 5  | time in 0 minutes, 4 seconds
	Loss: 0.0039(train)	|	Acc: 83.2%(train)
	Loss: 0.0002(valid)	|	Acc: 83.4%(valid)
Epoch: 6  | time in 0 minutes, 4 seconds
	Loss: 0.0037(train)	|	Acc: 84.5%(train)
	Loss: 0.0001(valid)	|	Acc: 86.7%(valid)
Epoch: 7  | time in 0 minutes, 4 seconds
	Loss: 0.0035(train)	|	Acc: 85.1%(train)
	Loss: 0.0001(valid)	|	Acc: 86.9%(valid)
Epoch: 8  | time in 0 minutes, 4 seconds
	Loss: 0.0034(train)	|	Acc: 86.6%(train)
	Loss: 0.0001(valid)	|	Acc: 87.4%(valid)
Epoch: 9  | time

### early stopping
* dev setでのaccuracyが4回連続で最高値を下回ったら訓練を終えることにする。
* early stoppingの実現については、PyTorch Lightningを使う手もある。
 * https://pytorch-lightning.readthedocs.io/en/latest/early_stopping.html

In [37]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

In [38]:
patience = 4
early_stop_count = 0
best_valid_acc = 0.0
valid_acc_threshold = 0.87

N_EPOCHS = 50 # エポック数を増やしておく
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs / 60
  secs = secs % 60

  print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
  print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
  print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

  # early stopping
  if best_valid_acc <= valid_acc:
    best_valid_acc = valid_acc
    early_stop_count = 0
  else:
    early_stop_count += 1
    if early_stop_count == patience:
      break

Epoch: 1  | time in 0 minutes, 4 seconds
	Loss: 0.0067(train)	|	Acc: 61.7%(train)
	Loss: 0.0002(valid)	|	Acc: 75.3%(valid)
Epoch: 2  | time in 0 minutes, 4 seconds
	Loss: 0.0051(train)	|	Acc: 75.6%(train)
	Loss: 0.0002(valid)	|	Acc: 82.5%(valid)
Epoch: 3  | time in 0 minutes, 4 seconds
	Loss: 0.0044(train)	|	Acc: 80.5%(train)
	Loss: 0.0002(valid)	|	Acc: 84.1%(valid)
Epoch: 4  | time in 0 minutes, 4 seconds
	Loss: 0.0040(train)	|	Acc: 83.1%(train)
	Loss: 0.0002(valid)	|	Acc: 83.6%(valid)
Epoch: 5  | time in 0 minutes, 4 seconds
	Loss: 0.0038(train)	|	Acc: 83.7%(train)
	Loss: 0.0001(valid)	|	Acc: 86.4%(valid)
Epoch: 6  | time in 0 minutes, 4 seconds
	Loss: 0.0036(train)	|	Acc: 84.8%(train)
	Loss: 0.0001(valid)	|	Acc: 86.0%(valid)
Epoch: 7  | time in 0 minutes, 4 seconds
	Loss: 0.0036(train)	|	Acc: 85.1%(train)
	Loss: 0.0001(valid)	|	Acc: 86.4%(valid)
Epoch: 8  | time in 0 minutes, 4 seconds
	Loss: 0.0033(train)	|	Acc: 86.8%(train)
	Loss: 0.0001(valid)	|	Acc: 87.6%(valid)
Epoch: 9  | time

## 07-05 テストセット上で評価
* 見つけ出したベストな設定を使って、テストセット上での最終的な評価をおこなう。

In [39]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_iterator, model, criterion)
print(f'\tLoss: {test_loss:.4f}(test)\t|\tAcc: {test_acc * 100:.1f}%(test)')

Checking the results of test dataset...
	Loss: 0.0000(test)	|	Acc: 88.2%(test)
